In [8]:
import numpy as np
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR100
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import datasets,transforms

In [9]:
def data_tf(x):
    x=np.array(x,dtype='float32')/255
    x=(x-0.5)/0.5
    x=transforms.ToTensor()(x)
    return x

In [10]:
train_set=CIFAR100('./data',train=True, transform=data_tf,download=True)
test_set=CIFAR100('./data',train=False, transform=data_tf,download=True)

Files already downloaded and verified
Files already downloaded and verified


In [13]:
from torch.utils.data import DataLoader
train_data = DataLoader(train_set, batch_size=64, shuffle=True)
test_data = DataLoader(test_set, batch_size=128, shuffle=False)

In [ ]:
class Alexnet(nn.Module):
    def __init__(self):                                        #inputsize=3*32*32
        super(Alexnet,self).__init__()                          
        self.featrue=nn.Sequential(
            nn.Conv2d(3,32,kernel_size=3,stride=1,padding=0),   ##outputsize=32*30*30
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),               ##outputsize=32*15*15          
            nn.Conv2d(32,96,kernel_size=3,stride=1,padding=0), ##outputsize=96*13*13  
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2),               ##outputsize=96*6*6  
        )
        self.classify=nn.Sequential(
            nn.Linear(96*6*6,1000),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1000,300),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(300,100),
        )
    def forward(self,x):
        x=self.featrue(x)
        x=x.view(x.size(0),-1)
        x=self.classify(x)
        return x

net=Alexnet()
print(net)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=1e-1,weight_decay=1e-4) # 使用随机梯度下降，学习率 0.1,正则化

In [ ]:
def get_acc(output, label):
    total = output.shape[0]    #output是二维的，零维度是batch，一维度是每个图片的预测标签
    _, pred_label = output.max(1)   #output.max(1)结果会返output一维度标签中最大的值和最大标签在一维度中的索引，这里只需用到索引
    num_correct = (pred_label == label).sum().item()    #.sum把前面那个张量中所有值加了起来，变成0维tensor，.item（）把它变为数值
    return num_correct / total

def set_learning_rate(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        
def train(net, train_data, valid_data, num_epochs, optimizer, criterion):
    if torch.cuda.is_available():
        net = net.cuda()
    for epoch in range(num_epochs):
        if epoch==10:
            set_learning_rate(optimizer,1e-2)
        train_loss = 0
        train_acc = 0
        net = net.train()
        for batchnum,(im, label) in enumerate(train_data):
            if torch.cuda.is_available():
                im = Variable(im.cuda())  
                label = Variable(label.cuda())  
            else:
                im = Variable(im)
                label = Variable(label)
            # forward
            output = net(im)
            loss = criterion(output, label)
            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # 计算误差
            train_loss += loss.item()
            train_acc += get_acc(output, label)

        if valid_data is not None:
            valid_loss = 0
            valid_acc = 0
            net = net.eval()
            for batchnum,(im, label) in enumerate(valid_data):
                if torch.cuda.is_available():
                    im = Variable(im.cuda(), volatile=True)
                    label = Variable(label.cuda(), volatile=True)
                else:
                    im = Variable(im, volatile=True)
                    label = Variable(label, volatile=True)
                # forward
                output = net(im)
                loss = criterion(output, label)
                # 计算误差
                valid_loss += loss.item()
                valid_acc += get_acc(output, label)
            epoch_str = ("Epoch %d. Train Loss: %f, Train Acc: %f, Valid Loss: %f, Valid Acc: %f, " % (epoch, train_loss / len(train_data),
                   train_acc / len(train_data), valid_loss / len(valid_data),valid_acc / len(valid_data)))
        else:
            epoch_str = ("Epoch %d. Train Loss: %f, Train Acc: %f, " %(epoch, train_loss / len(train_data),train_acc / len(train_data)))
        
        print(epoch_str)

In [ ]:
train(net,train_data,test_data,20,optimizer,criterion)